# 참고

- find_element(s)_by_class_name() -> find_element(s)(By.CLASS_NAME, '')로 활용 권장?
- By.에 따르는 CLASS_NAME, LINK_TEXT 등은 모두 대문자로 사용

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

import warnings
warnings.filterwarnings('ignore')

In [2]:
browser = webdriver.Chrome('C:/Users/Park/chromedriver.exe')
browser.maximize_window() # 크롬 창 오픈 후 전체화면으로 최대화

url = 'https://flight.naver.com/'
browser.get(url)

## 1. 항공권 날짜 선택 페이지로 넘어가기 위한 클릭

In [3]:
browser.find_element_by_xpath('//*[@id="__next"]/div/div[1]/div[4]/div/div/div[2]/div[2]/button[1]').click()

#### 선택된 날짜 지우기

In [6]:
browser.find_element(By.XPATH, '//*[@id="__next"]/div/div[1]/div[9]/div[2]/div[1]/button').click()

## 2. 날짜 선택

#### 방법 1. find_elements_by_link_text // find_element_by_class_name()

- 강의 당시와 다르게 웹페이지 구조가 변경되어 현재는 불가능함

In [10]:
browser.find_elements_by_link_text('27')[0].click()    # 27일 텍스트를 모두 찾아 0번째 인덱스 추출(이번달이 됨)
browser.find_elements_by_link_text('28')[0].click()

IndexError: list index out of range

#### 방법 2. find_element_by_xpath()

In [54]:
browser.find_element_by_xpath('//*[@id="__next"]/div/div[1]/div[9]/div[2]/div[1]/div[2]/div/div[2]/table/tbody/tr[3]/td[3]/button').click()
browser.find_element_by_xpath('//*[@id="__next"]/div/div[1]/div[9]/div[2]/div[1]/div[2]/div/div[2]/table/tbody/tr[3]/td[5]/button').click()

#### 방법 3. find_element_by_css_selector()

- 태그와 태그는 공백으로 구분(>로도 가능하네)
- 클래스 네임과 같은 것은 .으로 구분
- 예 : ('table tbody tr') (sc-breuTD.ehtLKD.txt)
- css_selector()에서만 이런 것인지는 불확실함

In [4]:
# 달력 전체 중 월 요소 추출
month = browser.find_elements_by_css_selector('div.sc-jqUVSM.ilolms.awesome-calendar div.sc-kDDrLX.ctbFvd.month')
# 월의 첫번째, 즉 이번달 요소 추출
week = month[0].find_elements_by_css_selector('table tbody tr')
# 이번달의 5번째(4) 주 추출
go_day = week[4].find_elements_by_css_selector('td')
# 5번째 주의 4번째 일 추출(작성 기준일로 2022-04-11)
go_day[3].click()

back_day = week[4].find_elements_by_css_selector('td')
back_day[6].click()

In [16]:
# 이번달 27일부터 다음달 4일로 추출해보자
month = browser.find_elements_by_css_selector('div.sc-jqUVSM.ilolms.awesome-calendar div.sc-kDDrLX.ctbFvd.month')

go_day = month[0].find_elements_by_css_selector('table tbody tr')[4].find_elements_by_css_selector('td')
go_day[3].click()

back_day = month[1].find_elements_by_css_selector('table tbody tr')[0].find_elements_by_css_selector('td')
back_day[3].click()

In [4]:
# 바로 위의 코드와 동일함
month = browser.find_elements(By.CSS_SELECTOR,'div.sc-jqUVSM.ilolms.awesome-calendar div.sc-kDDrLX.ctbFvd.month')

go_day = month[0].find_elements(By.CSS_SELECTOR,'table tbody tr')[4].find_elements(By.CSS_SELECTOR,'td')
go_day[3].click()

back_day = month[1].find_elements(By.CSS_SELECTOR,'table tbody tr')[0].find_elements(By.CSS_SELECTOR,'td')
back_day[3].click()

## 3. 공항 선택

- 출발 : 인천공항
- 도착 : 호놀룰루 공항

#### 출발지 선택

- 각 선택별 두 번째 반복문에서 왜 첫번째처럼 바로 .click()할 경우 오류가 생기는지 모르겠음...

In [5]:
browser.find_element_by_xpath('//*[@id="__next"]/div/div[1]/div[4]/div/div/div[2]/div[1]/button[1]/b').click()

# css_selector로 버튼 요소 추출
country = browser.find_elements(By.CSS_SELECTOR, 'div.autocomplete_content__3RhAZ section section button')

for i in range(len(country)):
    if country[i].text == '국내':
        country[i].click()
        
go_port = browser.find_elements(By.CSS_SELECTOR, 'div.autocomplete_list__de1dI button')

for i in range(len(go_port)):
    if go_port[i].text.split(',')[0] == '인천':
        a = go_port[i]
        
a.click()

#### 도착지 선택

In [6]:
browser.find_element_by_xpath('//*[@id="__next"]/div/div[1]/div[4]/div/div/div[2]/div[1]/button[2]/b').click()

country = browser.find_elements(By.CSS_SELECTOR, 'div.autocomplete_content__3RhAZ section section button')

for i in range(len(country)):
    if country[i].text == '미주':
        country[i].click()
        
arrive = browser.find_elements(By.CSS_SELECTOR, 'div.autocomplete_list__de1dI button')

for i in range(len(arrive)):
    if '호놀룰루' in arrive[i].text:
        b = arrive[i]
        
b.click()

## 3. 항공권 검색

In [26]:
browser.find_element(By.XPATH, '//*[@id="__next"]/div/div[1]/div[4]/div/div/button/span').click()

## 4. 검색 결과 출력

- 만일 전체 코드를 한번에 실행할 경우 항공권 검색 딜레이가 존재하기 때문에 결과 출력이 안될 수 있다.
- 따라서 WebDriverWait으로 코드 동작이 완료될 때까지 다음 코드 동작을 제어하는 코드를 작성한다.

In [24]:
# 참고
# result = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.concurrent_ConcurrentList__1EKaB div')))

In [281]:
result = browser.find_elements(By.CSS_SELECTOR, 'div.concurrent_ConcurrentList__1EKaB div.concurrent_ConcurrentItemContainer__2lQVG.result')

go = []
back = []
time_1 =[]
time_2 =[]
time_3 =[]
time_4 =[]
price =[]


for i in range(100):
    
    flight_area = result[i].find_elements(By.CLASS_NAME, 'airline')
    flight_time = result[i].find_elements(By.CLASS_NAME, 'route_time__-2Z1T')
    flight_price = result[i].find_element(By.CLASS_NAME, 'item_fare__3YGTI')
    
    go.append(flight_area[0].text)
    if len(flight_area) == 2:
        back.append(flight_area[1].text)
    else:
        back.append('none')

    for idy in range(len(flight_time)):
        if idy%4 == 0:
            time_1.append(flight_time[idy].text)
        elif idy%4 == 1:
            time_2.append(flight_time[idy].text)
        elif idy%4 == 2:
            time_3.append(flight_time[idy].text)
        else:
            time_4.append(flight_time[idy].text)
        
    price.append(flight_price.text)

In [288]:
final = pd.DataFrame([go,time_1,time_2, back,time_3,time_4]).T
final.columns = ['출국 항공사', '출국/시간',' 출국/도착', '입국 항공사','입국/출발','입국/도착']
final.to_csv('C:/Users/Park/_Python_data/airline_tiket_100.csv', encoding = 'utf-8-sig')
final

,출국 항공사,출국/시간,출국/도착,입국 항공사,입국/출발,입국/도착
0,아시아나항공,20:20,10:10,none,11:50,17:10
1,대한항공,20:35,09:45,none,13:00,18:05
2,유나이티드항공하와이안항공 공동운항,16:50,06:38,none,17:10,14:55
3,유나이티드항공하와이안항공 공동운항,16:50,16:17,none,17:10,14:55
4,유나이티드항공하와이안항공 공동운항,16:50,06:38,none,11:58,14:55
...,...,...,...,...,...,...
95,여러항공사,20:40,05:10,none,13:30,05:00
96,여러항공사,20:40,05:10,none,12:05,05:00
97,여러항공사,20:40,05:10,none,07:50,05:00
98,여러항공사,20:20,10:10,none,14:55,17:20


# 전체 코드

In [2]:
browser = webdriver.Chrome('C:/Users/Park/chromedriver.exe')
browser.maximize_window()
url = 'https://flight.naver.com/'
browser.get(url)

browser.find_element_by_xpath('//*[@id="__next"]/div/div[1]/div[4]/div/div/div[2]/div[2]/button[1]').click()
month = browser.find_elements(By.CSS_SELECTOR,'div.sc-jqUVSM.ilolms.awesome-calendar div.sc-kDDrLX.ctbFvd.month')
go_day = month[0].find_elements(By.CSS_SELECTOR,'table tbody tr')[4].find_elements(By.CSS_SELECTOR,'td')
go_day[3].click()
back_day = month[1].find_elements(By.CSS_SELECTOR,'table tbody tr')[0].find_elements(By.CSS_SELECTOR,'td')
back_day[3].click()

browser.find_element_by_xpath('//*[@id="__next"]/div/div[1]/div[4]/div/div/div[2]/div[1]/button[1]/b').click()
country = browser.find_elements(By.CSS_SELECTOR, 'div.autocomplete_content__3RhAZ section section button')
for i in range(len(country)):
    if country[i].text == '국내':
        country[i].click()        
go_port = browser.find_elements(By.CSS_SELECTOR, 'div.autocomplete_list__de1dI button')
for i in range(len(go_port)):
    if go_port[i].text.split(',')[0] == '인천':
        a = go_port[i]        
a.click()

browser.find_element_by_xpath('//*[@id="__next"]/div/div[1]/div[4]/div/div/div[2]/div[1]/button[2]/b').click()
country = browser.find_elements(By.CSS_SELECTOR, 'div.autocomplete_content__3RhAZ section section button')
for i in range(len(country)):
    if country[i].text == '미주':
        country[i].click()       
arrive = browser.find_elements(By.CSS_SELECTOR, 'div.autocomplete_list__de1dI button')
for i in range(len(arrive)):
    if '호놀룰루' in arrive[i].text:
        b = arrive[i]        
b.click()

browser.find_element(By.XPATH, '//*[@id="__next"]/div/div[1]/div[4]/div/div/button/span').click()
time.sleep(10)

result = browser.find_elements(By.CSS_SELECTOR, 'div.concurrent_ConcurrentList__1EKaB div.concurrent_ConcurrentItemContainer__2lQVG.result')
go = []
back = []
time_1 =[]
time_2 =[]
time_3 =[]
time_4 =[]
price =[]
for i in range(100):    
    flight_area = result[i].find_elements(By.CLASS_NAME, 'airline')
    flight_time = result[i].find_elements(By.CLASS_NAME, 'route_time__-2Z1T')
    flight_price = result[i].find_element(By.CLASS_NAME, 'item_fare__3YGTI')    
    go.append(flight_area[0].text)
    if len(flight_area) == 2:
        back.append(flight_area[1].text)
    else:
        back.append('none')
    for idy in range(len(flight_time)):
        if idy%4 == 0:
            time_1.append(flight_time[idy].text)
        elif idy%4 == 1:
            time_2.append(flight_time[idy].text)
        elif idy%4 == 2:
            time_3.append(flight_time[idy].text)
        else:
            time_4.append(flight_time[idy].text)        
    price.append(flight_price.text)
    
final = pd.DataFrame([go,time_1,time_2, back,time_3,time_4]).T
final.columns = ['출국 항공사', '출국/시간',' 출국/도착', '입국 항공사','입국/출발','입국/도착']
final.to_csv('C:/Users/Park/_Python_data/airline_tiket_100.csv', encoding = 'utf-8-sig')
final

,출국 항공사,출국/시간,출국/도착,입국 항공사,입국/출발,입국/도착
0,아시아나항공,20:20,10:10,none,11:50,17:10
1,대한항공,20:35,09:45,none,13:00,18:05
2,유나이티드항공하와이안항공 공동운항,16:50,06:38,none,17:10,14:55
3,유나이티드항공하와이안항공 공동운항,16:50,16:17,none,17:10,14:55
4,유나이티드항공하와이안항공 공동운항,16:50,13:25,none,17:10,14:55
...,...,...,...,...,...,...
95,대한항공하와이안항공 공동운항,21:25,10:55,델타항공대한항공 공동운항,12:35,17:50
96,여러항공사,20:40,21:00,none,21:20,17:35
97,여러항공사,20:40,21:00,none,14:30,17:35
98,유나이티드항공하와이안항공 공동운항,16:50,16:17,none,13:40,14:55
